In [1]:
from pathlib import Path
from scipy.io import loadmat

import pandas as pd


from reciprocal_isomap import ReciprocalIsomap

import numpy as np 
from kmapper import KeplerMapper
from landmark_cover import LandmarkCover
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt

from nilearn.connectome import ConnectivityMeasure

In [3]:
#Load one patient data as X = time x ROI


base_dir = Path(os.getcwd()).parent 
data_dir = base_dir / 'data'
data = loadmat(data_dir / 'timeseries_allsubs.mat')
# subs, timepoints, regions
data['timeseries'].shape

X = data['timeseries'][7]

print(X.shape)

#Generate encoding matrix of timing labels

labels = loadmat(data_dir / 'labels.mat')
labels = [e[0] for e in labels['labels'].squeeze()]

df = pd.DataFrame(labels, columns=['labels'])
target, labels = pd.factorize(df.labels.values)
y = pd.DataFrame({l:(target==i).astype(int) for i,l in enumerate(labels)})

(4340, 100)


In [4]:
# compute distances with reciprocal isomap


# compute geodesic distances on a reciprocal neighbor graph
r_isomap = ReciprocalIsomap(n_neighbors=30,neighbors_mode="distance", n_jobs=-2)
geodesic_distances = r_isomap._reciprocal_distances(X).toarray()


/home/poetz/anaconda3/envs/tda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function graph_shortest_path is deprecated; `graph_shortest_path` is deprecated in 1.0 (renaming of 0.25) and will be removed in 1.2. Use `scipy.sparse.csgraph.shortest_path` instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
4340/4

1085.0

In [13]:
#feed distances to KeplerMapper using Landmark Cover



# setup KeplerMapper object
mapper = KeplerMapper(verbose=1)

# run KeplerMapper using the LandmarkCover
graph = mapper.map(
    lens=geodesic_distances, X=X,
    cover=LandmarkCover(n_landmarks=500, perc_overlap=0.15, metric='precomputed'),
    clusterer=AgglomerativeClustering(n_clusters=10, linkage='single'),
    remove_duplicate_nodes=True,
)

# visualize the graph
html = mapper.visualize(graph, path_html=f'last_patient_7_500land.html')

KeplerMapper()
Mapping on data shaped (4340, 100) using lens shaped (4340, 4340)

Creating 500 hypercubes.
Merged 2168 duplicate nodes.

Number of nodes before merger: 4860; after merger: 2692


Created 11857 edges and 2692 nodes in 0:01:00.463291.
Wrote visualization to: last_patient_7_500land.html


In [12]:
## Visualize the shape graph using DyNeuSR's DyNeuGraph
from dyneusr import DyNeuGraph

dG = DyNeuGraph(G=graph, y=y)
dG.visualize('patient7_dyneusr_02.html')

label,group,value,row_count
REST,0,2400,4340
EMOTION,1,176,4340
GAMBLING,2,253,4340
LANGUAGE,3,316,4340
MOTOR,4,284,4340
RELATIONAL,5,232,4340
SOCIAL,6,274,4340
MEMORY,7,405,4340


Already serving localhost:None 
[Force Graph] file:///home/poetz/Desktop/PhD/Projects/Projects%20CSSS/TDA/CSSS22-TDA/code/patient7_dyneusr_02.html


DyNeuGraph()